# 5 SVM 课堂练习

姓名：华勤达

学号：2018212212229

贷款违约数据集来自Kaggle数据科学竞赛平台2011年的一个竞赛项目“Give Me Some Cerdit”，该数据集共包含25000个样本，其中150000个样本作为训练集，100000个样本用作测试集。

该训练集共有150000条借款人的历史数据，其中违约样本10026条，占样本总量的6.684%，贷款违约率为6.684%，未违约样本139974条，占样本总量的93.316%。可以看出该数据集是一个典型的高度不平衡的数据。数据集中包括了借款人的年龄、收入、家庭等情况和贷款情况，共11个变量，其中SeriousDlqin2yrs是label标签，另外10个变量为预测特征。下表列出了变量名称及数据类型情况： 

表1 数据集变量情况

| 变量名 | 变量描述 | 类型 |
|  - | -  | -|
|SeriousDlqin2yrs | 是否违约 | Y/N|
|RevolvingUtilizationOfUnsecuredLines | 信用卡和个人信用贷款的总额（不含房贷、类似车贷的分期付款等）除以授信额度之和 | 百分比 |
|age | 借款人年龄 | 整数 |
|NumberOfTime30-59DaysPastDueNotWorse | 过去两年中借款人逾期30-59天的次数 | 整数 |
|DebtRatio | 每月债务偿还数、赡养费、生活成本等除以每月总收入 | 百分比 |
|MonthlyIncome | 月收入 | 实数|
|NumberOfOpenCreditLinesAndLoans | 开放贷款（Open loans，分期付款如车贷和抵押）和信用贷款（Lines of credit,如信用卡）的数量 |整数 |
|NumberOfTimes90DaysLate| 过去两年中借款人逾期90天及以上的次数 | 整数|
|NumberRealEstateLoansOrLines | 抵押和房地产贷款含房屋抵押式信用贷款的次数 | 整数|
|NumberOfTime60-89DaysPastDueNotWorse| 过去两年中借款人逾期60-89天的次数 | 整数 |
|NumberOfDependents | 不包括本人在内家庭中需要抚养的人（配偶及子女等等）数 | 整数 |

1. 数据导入和清洗

In [1]:
import pandas as pd

train_data = pd.read_csv('cs-training(2).csv')

feature_names = ['个人信用贷款/授信额度','借款人年龄','逾期30-59天次数','债务收入比','贷款数量']
drop_features = ['Unnamed: 0','MonthlyIncome','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines',
                 'NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
target_names = ['正常','违约']

train_data = train_data.drop(drop_features, axis=1).dropna()
X = train_data.drop(["SeriousDlqin2yrs"], axis=1).copy()
y = train_data.SeriousDlqin2yrs
X.shape, y.shape

((150000, 5), (150000,))

2. 分训练集和测试集

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                   X, y, test_size=0.2, random_state=42)
X_train.shape,X_test.shape

((120000, 5), (30000, 5))

3. 采用RandomizedSearchCV搜索最佳gamma和C的参数

In [3]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

#定义SVC对象
my_svc = SVC()


#parameters字典，包括对C和对gamma的定义
parameters = {'C':[1, 2], 'gamma':[0.125, 0.5]}
 
# 构建pipeline,scoring='f1',cv=5


# svr = RandomizedSearchCV(my_svc, parameters, scoring='f1', cv=5)

pipe_svc_rnd = Pipeline([('sc', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', RandomizedSearchCV(my_svc, parameters, scoring='f1', cv=5,n_jobs=-1))
                    ])
pipe_svc_rnd.fit(X_train, y_train)



/Users/sunrise/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Pipeline(steps=[('sc', StandardScaler()), ('pca', PCA(n_components=2)),
                ('clf',
                 RandomizedSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
                                    param_distributions={'C': [1, 2],
                                                         'gamma': [0.125, 0.5]},
                                    scoring='f1'))])

In [4]:
%time pipe_svc_rnd.fit(X_train, y_train)

/Users/sunrise/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: user 1min 11s, sys: 429 ms, total: 1min 12s
Wall time: 16min 23s


Pipeline(steps=[('sc', StandardScaler()), ('pca', PCA(n_components=2)),
                ('clf',
                 RandomizedSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
                                    param_distributions={'C': [1, 2],
                                                         'gamma': [0.125, 0.5]},
                                    scoring='f1'))])

In [6]:
print('--------结果报告----------')
print('最优模型(best_estimator_):',pipe_svc_rnd['clf'].best_estimator_) 
print('最优参数(best_params_):',pipe_svc_rnd['clf'].best_params_) 
print('best_score_:',pipe_svc_rnd['clf'].best_score_)
print('scorer_:',pipe_svc_rnd['clf'].scorer_)
print('-------------------------')

--------结果报告----------
最优模型(best_estimator_): SVC(C=1, gamma=0.125)
最优参数(best_params_): {'gamma': 0.125, 'C': 1}
best_score_: 0.021101264740652114
scorer_: make_scorer(f1_score, average=binary)
-------------------------


In [7]:
means = pipe_svc_rnd['clf'].cv_results_['mean_test_score']
params = pipe_svc_rnd['clf'].cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

0.021101  with:   {'gamma': 0.125, 'C': 1}
0.019683  with:   {'gamma': 0.5, 'C': 1}
0.020614  with:   {'gamma': 0.125, 'C': 2}
0.020162  with:   {'gamma': 0.5, 'C': 2}


In [8]:
#检查模型准确率
from sklearn.metrics import accuracy_score

%time y_pred = pipe_svc_rnd.predict(X_test)

CPU times: user 43.1 s, sys: 91.8 ms, total: 43.2 s
Wall time: 42.8 s


In [9]:
#查看混淆矩阵
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[28018,    26],
       [ 1928,    28]])

In [10]:
#查看F1score
from sklearn import metrics

score = metrics.f1_score(y_test,y_pred)
score

0.027860696517412933